# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.110158e+02     1.541306e+00
 * time: 0.07048702239990234
     1     1.116624e+01     8.587393e-01
 * time: 1.2871170043945312
     2    -1.150379e+01     1.024068e+00
 * time: 1.3819541931152344
     3    -3.370710e+01     8.014912e-01
 * time: 1.5071361064910889
     4    -4.723906e+01     5.713751e-01
 * time: 1.6329150199890137
     5    -5.692786e+01     2.323288e-01
 * time: 1.7645761966705322
     6    -5.985872e+01     1.660531e-01
 * time: 1.8509631156921387
     7    -6.097201e+01     5.586167e-02
 * time: 1.938735008239746
     8    -6.141811e+01     4.638205e-02
 * time: 2.0263760089874268
     9    -6.166949e+01     3.671222e-02
 * time: 2.1284801959991455
    10    -6.186315e+01     2.615150e-02
 * time: 2.2166030406951904
    11    -6.199789e+01     1.838454e-02
 * time: 2.3046281337738037
    12    -6.207127e+01     1.784112e-02
 * time: 2.392592191696167
    13    -6.211207e+01     1.603624e-02
 * time: 2.47913408279

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671058
    AtomicLocal         -18.8557626
    AtomicNonlocal      14.8522615
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485352 
    Xc                  -19.3336813

    total               -62.261666456609
